In [1]:
#@title Cell 1: Setup & Imports

import torch
import torch.nn as nn
import torch.nn.functional as F
import math, os, time

print("PyTorch version:", torch.__version__)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)


PyTorch version: 2.9.0+cu126
Using device: cuda


In [2]:
#@title Cell 2: Download Tiny Shakespeare text dataset

# This will download input.txt (tiny-shakespeare) from Karpathy's repo
if not os.path.exists("input.txt"):
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O input.txt

with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Dataset length (characters):", len(text))
print("Sample:\n", text[:500])


--2025-12-09 08:52:31--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.006s  

2025-12-09 08:52:32 (165 MB/s) - ‘input.txt’ saved [1115394/1115394]

Dataset length (characters): 1115394
Sample:
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a 

In [3]:
#@title Cell 3: Build vocabulary, encoder/decoder, and split data

# Get all unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Vocab size:", vocab_size)
print("Chars:", ''.join(chars))

# Mapping from char to int and back
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def encode(s: str):
    """String -> list of ints (token ids)"""
    return [stoi[c] for c in s]

def decode(l):
    """List of ints -> string"""
    return ''.join([itos[i] for i in l])

# Convert entire dataset to a tensor of ints
data = torch.tensor(encode(text), dtype=torch.long)
print("Data shape:", data.shape)

# Train/Validation split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print("Train size:", train_data.shape, "Val size:", val_data.shape)


Vocab size: 65
Chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Data shape: torch.Size([1115394])
Train size: torch.Size([1003854]) Val size: torch.Size([111540])


In [4]:
#@title Cell 4: Define hyperparameters and get_batch() helper

batch_size   = 64    # how many sequences per batch
block_size   = 128   # max context length for the model
max_iters    = 2000  # you can increase this (e.g. 5000–10000) for better quality
eval_interval = 200
learning_rate = 3e-4
eval_iters   = 200  # how many batches to average for loss estimate

torch.manual_seed(1337)

def get_batch(split):
    """Generate a batch of inputs (x) and targets (y) for training."""
    data_split = train_data if split == "train" else val_data
    ix = torch.randint(len(data_split) - block_size, (batch_size,))
    x = torch.stack([data_split[i:i+block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Quick sanity check
xb, yb = get_batch('train')
print("Batch x shape:", xb.shape)  # (B, T)
print("Batch y shape:", yb.shape)  # (B, T)


Batch x shape: torch.Size([64, 128])
Batch y shape: torch.Size([64, 128])


In [5]:
#@title Cell 5: Define the mini-GPT transformer model

n_embed = 384   # embedding dimension
n_head  = 6     # number of attention heads
n_layer = 6     # number of transformer blocks
dropout = 0.2

class Head(nn.Module):
    """One head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)

        # compute attention scores
        wei = q @ k.transpose(-2, -1) / math.sqrt(k.size(-1))  # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # weighted sum of values
        v = self.value(x)  # (B, T, head_size)
        out = wei @ v      # (B, T, head_size)
        return out

class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj  = nn.Linear(num_heads * head_size, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    """Simple MLP with one hidden layer"""

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Transformer block: communication (attention) then computation (MLP)"""

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # token and position embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        # transformer blocks
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)  # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # token + positional embeddings
        tok_emb = self.token_embedding_table(idx)                   # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))  # (T, C)
        x = tok_emb + pos_emb                                       # (B, T, C)

        # transformer blocks
        x = self.blocks(x)                                          # (B, T, C)
        x = self.ln_f(x)                                            # (B, T, C)
        logits = self.lm_head(x)                                    # (B, T, vocab_size)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        """Autoregressively generate new tokens from the model."""
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]  # crop to last block_size tokens
            logits, _ = self(idx_cond)       # (B, T, vocab_size)
            logits = logits[:, -1, :]        # take the last time step
            probs = F.softmax(logits, dim=-1)  # (B, vocab_size)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # append
        return idx

model = GPTLanguageModel().to(device)
print("Model parameters:", sum(p.numel() for p in model.parameters()) / 1e6, "M")


Model parameters: 10.739777 M


In [6]:
#@title Cell 6: Loss evaluation helper (train & val)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = []
        for _ in range(eval_iters):
            xb, yb = get_batch(split)
            _, loss = model(xb, yb)
            losses.append(loss.item())
        out[split] = sum(losses) / len(losses)
    model.train()
    return out


In [7]:
#@title Cell 7: Train the model

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

start_time = time.time()

for iter in range(max_iters):

    # Occasional evaluation
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        elapsed = time.time() - start_time
        print(f"Step {iter:4d}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}  (elapsed {elapsed:.1f}s)")

    # Get batch
    xb, yb = get_batch('train')

    # Forward
    logits, loss = model(xb, yb)
    # Backward
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("Training finished.")


Step    0: train loss 4.3336, val loss 4.3296  (elapsed 27.9s)
Step  200: train loss 2.2694, val loss 2.2926  (elapsed 103.7s)
Step  400: train loss 1.8955, val loss 1.9856  (elapsed 179.0s)
Step  600: train loss 1.7057, val loss 1.8554  (elapsed 254.6s)
Step  800: train loss 1.5995, val loss 1.7765  (elapsed 329.8s)
Step 1000: train loss 1.5203, val loss 1.7114  (elapsed 405.2s)
Step 1200: train loss 1.4683, val loss 1.6730  (elapsed 480.4s)
Step 1400: train loss 1.4346, val loss 1.6431  (elapsed 555.4s)
Step 1600: train loss 1.3986, val loss 1.6134  (elapsed 630.3s)
Step 1800: train loss 1.3679, val loss 1.5884  (elapsed 705.1s)
Step 1999: train loss 1.3468, val loss 1.5765  (elapsed 779.9s)
Training finished.


In [8]:
#@title Cell 8: Generate text from an empty prompt

model.eval()
start_context = torch.zeros((1, 1), dtype=torch.long, device=device)  # start token = 0
with torch.no_grad():
    generated = model.generate(start_context, max_new_tokens=500)

print(decode(generated[0].tolist()))



Mether:
Mine man, that woll I envious sing them;
Forgelow's absence, for thou didst answer that,
Be know a kneat cain the bobe bare hateful life;
Lest him hath dine to here will his poor so--
And that thou hast are that thou comes be a courten?

First Citizen:
Why, sworn, I would we about not from to this pirce should
witch heir dues and revenge to it: he'll, we in
Be brook to the noble find, and purch's heart with says
Alandly deads, as it such from the viyors
Maketage it aland even death. This


In [9]:
#@title Cell 9: Prompt-based generation + interactive loop

model.eval()

def generate_from_prompt(prompt: str, max_new_tokens: int = 200) -> str:
    """
    Take a text prompt (string), run it through the model,
    and return the generated continuation (string).
    """
    if len(prompt.strip()) == 0:
        # behave like empty context
        idx = torch.zeros((1, 1), dtype=torch.long, device=device)
    else:
        encoded = encode(prompt)
        idx = torch.tensor([encoded], dtype=torch.long, device=device)

    with torch.no_grad():
        out = model.generate(idx, max_new_tokens=max_new_tokens)

    return decode(out[0].tolist())


# 1) Quick test with a fixed prompt
test_prompt = "ROMEO:"
print("=== Test prompt ===")
print("Prompt:", repr(test_prompt))
print("Output:\n")
print(generate_from_prompt(test_prompt, max_new_tokens=300))


# 2) Optional: interactive loop (works in Colab terminal/console cell)
print("\n--- Interactive mode ---")
print("Type a prompt and press Enter. Type 'quit' to stop.\n")

while True:
    user_inp = input("Your prompt: ")
    if user_inp.lower() in {"quit", "exit", "q"}:
        print("Bye!")
        break

    out_text = generate_from_prompt(user_inp, max_new_tokens=250)
    print("\nModel output:\n")
    print(out_text)
    print("\n" + "-" * 80 + "\n")


=== Test prompt ===
Prompt: 'ROMEO:'
Output:

ROMEO:
I am, no, who that friar, I speak with the plimment
In thy goodness, this flance, time arms carqued
From hindered, and Marcius and wretched by night.

DUKE OF YORK:
Ay, shall how so us.

LUCIO:
I turn, sir, and you
Is as the feel's cocking and Here's hand:
O, my wrong is Camillo made to a privillow

--- Interactive mode ---
Type a prompt and press Enter. Type 'quit' to stop.

Your prompt: that's beautiful evening 

Model output:

that's beautiful evening man;
That I may if mock'd my fool with go conquarl.

DUKE VINCENTIO:
Nay, have you bids, and I follow us gentrans she
Soul, I should best were every you are a snab
Through of find you.

QUEEN ELIZABETH:
O, gogua O: we do, forget, for my hope.

GRERD 

--------------------------------------------------------------------------------

Your prompt: science

Model output:

science have fellow'd you.
The cendicious heart condemned
If his drink din his partue proaply;
And should me not by f